# Auto Sync Google Sheets
Run the notebook then authorize.

In [ ]:

!pip install gspread pandas gspread-dataframe --quiet

import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# List of source links
source_links = [
    "LINK_1",
    "LINK_2",
]

target_sheet_id = "12JcDBZTcq7HQP3XtL4bAzpFfwb0wwkrQJDWdN0Erxvw"
target_sheet_name = "TONG_HOP"

def read_first_sheet(url):
    try:
        file_id = url.split("/d/")[1].split("/")[0]
        sh = gc.open_by_key(file_id)
        ws = sh.get_worksheet(0)
        df = pd.DataFrame(ws.get_all_records())
        df["Nguon"] = url
        return df
    except Exception as e:
        print("Error:", url, e)
        return pd.DataFrame()

all_data = pd.DataFrame()
for link in source_links:
    df = read_first_sheet(link)
    if not df.empty:
        all_data = pd.concat([all_data, df], ignore_index=True)

target_file = gc.open_by_key(target_sheet_id)
try:
    ws = target_file.worksheet(target_sheet_name)
except:
    ws = target_file.add_worksheet(target_sheet_name, 1000, 20)

ws.clear()
set_with_dataframe(ws, all_data)

print("DONE")
